## BayArea Data Exploration

Arezoo Besharati, UrbanSim, June 2018 


In [1]:
import os; os.chdir('../')
import numpy as np, pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import skew
import warnings;
warnings.simplefilter('ignore')

In [2]:
class color:
    BOLD = '\033[1m'
    END = '\033[0m'

In [3]:
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
import orca

### Load data

In [4]:
# Load any script-based Orca registrations
from scripts import datasources
from scripts import models

In [5]:
# get a list of all existing tables
orca.list_tables()

['parcels', 'buildings', 'rentals', 'units', 'households', 'persons', 'jobs']

In [ ]:
# make dataframes for all tables
frames = {name: orca.get_table(name).to_frame() for name in orca.list_tables()}

### Generate Node variables

In [8]:
orca.run(['initialize_network'])

Running step 'initialize_network'
Time to execute step 'initialize_network': 18.35 s
Total time to execute iteration 1 with iteration value None: 18.36 s


In [9]:
orca.run(['network_aggregations'])

Running step 'network_aggregations'
Computing accessibility variables
Computing residential_units_500
Removed 4 rows because they contain missing values
Computing residential_units_1500
Removed 4 rows because they contain missing values
Computing population_1500
Removed 189769 rows because they contain missing values
Computing poor_1500
Removed 53114 rows because they contain missing values
Computing renters_1500
Removed 102597 rows because they contain missing values
Computing med_income_500
Removed 189769 rows because they contain missing values
Computing med_income_500_log1p
Removed 189769 rows because they contain missing values
Computing med_income_500_log
Removed 189769 rows because they contain missing values
Computing job_1500
Computing med_rent_per_sqft_log
Computing med_rent_per_sqft
Computing ave_hhsize
Removed 189769 rows because they contain missing values
       residential_units_500  residential_units_1500  population_1500  \
count           30826.000000            30826

In [ ]:
# for table_name in orca.list_tables():
#     print(table_name.upper())
#     print(orca.get_table(table_name).to_frame().columns.tolist())
#     print()

## Data Cleaning

 - Handling missing values
 - Checking the data types
 - Normalization: check for feature's distributions (the skewness)
 - Scalling: Check the units of data
 - Create dummy variables if needed


## 1. Buildings Table

### 1. 1. Missing values and data types

In [ ]:
# Does data have missing values? 
bld = orca.get_table('buildings').to_frame()
print(color.RED + 'Data has missing values'+ color.END if bld.isnull().values.any() else 'Data doesnt have missing values')
print ('')

# What columns/ fields/ variables
print (color.BOLD +'Columns with missing values :\n'+ color.END + '{}'.format(bld.columns[bld.isna().any()].tolist()))
print ('')

# Check features datatypes and see if there is any string feature that needs to be int/float
print(color.BOLD + 'String Features: \n'+ color.END + '{}'.format(bld.dtypes[bld.dtypes == "object"]))


costar_rent should be integer. 

In [ ]:
# change the data type and update the column
bld.costar_rent = pd.to_numeric(bld.costar_rent, errors='coerce')

# Take a count of missing values. What proportion of the data is missing? 
missing_values_count_bld = bld[['redfin_sale_price', 'redfin_sale_year', 'redfin_home_type', 'costar_property_type', 'costar_rent', 'node_id']].isnull().sum()
print ('')
print(color.BOLD + 'BUILDINGS' + color.END + '\nCount of missing points: \n{}'.format (missing_values_count_bld))
print('Proportion of missing points: \n{}'.format (missing_values_count_bld/ len (bld.costar_rent)))
print ('')


- 99% of costar rent is missing!! Don't use this feature
- 40 percent of redfin sale price and year is missing  

In [ ]:
# drop the missing rows for "redfin_home_type" and "costar_property_type"       
bld.dropna(subset=['redfin_home_type', 'costar_property_type'],inplace = True)

## Update the whole table
orca.add_table('buildings', bld)

In [ ]:
#bld.redfin_home_type.unique()

In [ ]:
#bld = bld[(bld.redfin_home_type !='Ranch')&(bld.redfin_home_type !='Timeshare')&(bld.redfin_home_type !='Other')& (bld.redfin_home_type !='Unknown')&(bld.redfin_home_type !='')]
## Update the whole table
#orca.add_table('buildings', bld)

### 1. 2. Normalization

In [ ]:
numeric_feats = bld.dtypes[bld.dtypes != "object"].index

skewed_feats = bld[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 1]
print(color.BOLD +'Skewed features are as follows'+ color.END + '\n{}'.format (skewed_feats))

In [ ]:
skewed_feats = skewed_feats[['improvement_value', 'residential_sqft', 'sqft_per_unit','non_residential_sqft',
                            'building_sqft','nonres_rent_per_sqft','res_price_per_sqft','redfin_sale_price']]

In [ ]:
skewed_feats = skewed_feats.index

bld[skewed_feats] = np.log1p(bld[skewed_feats])

# Update the building table 
orca.add_table('buildings', bld)

_____________________________________________________________________________________________________

## 2. Households Table

### 2. 1. Missing values and data types

In [ ]:
# Does data have missing values? 
households= orca.get_table('households').to_frame()
print(color.RED + 'Data has missing values'+ color.END if households.isnull().values.any() else 'Data doesnt have missing values')
print ('')

# What columns/ variables
print ('Columns with missing values : {}'.format(households.columns[households.isna().any()].tolist()))
print ('')

# Check features datatypes and see if there is any string feature that needs to be int/float
print(color.BOLD + 'String Features: \n'+ color.END + '{}'.format(households.dtypes[households.dtypes == "object"]))

In [ ]:
# Take a count of missing values. What proportion of the data is missing? 
missing_values_count_hh = households[['workers', 'children', 'node_id']].isnull().sum()
print ('')
print(color.BOLD + 'HOUSEHOLDS' + color.END + '\nCount of missing points: \n{}'.format (missing_values_count_hh))
print('Proportion of missing points: \n{}'.format (missing_values_count_hh/ len (households.node_id)))

### Handling missing values

#### Option 1: Not using the features with missing values !

#### Option 2: Deleting/ignoring the missing values !!!

In [ ]:
##I don't recommend this method at all. But in case one wants to do it 

#households.dropna(axis=0, how='any', inplace = True)

## Update the whole table
#orca.add_table('households', households)

#### Option 3:  Filling in the Value

In [ ]:
# Impute with mean 
households.workers.fillna(households.workers.mean(), inplace = True)

# Update column
orca.get_table('households').update_col_from_series('workers', households.workers)

### 2. 2. Normalizing

In [ ]:
# Only income needs to be normalized
households.income = np.log1p(households.income)

# Update column
orca.get_table('households').update_col_from_series('income', households.income)

_____________________________________________________________________________________________________

## 3. Parcels Table

In [ ]:
# Does data have missing values? 
parcels= orca.get_table('parcels').to_frame()
print(color.RED + 'Data has missing values'+ color.END if parcels.isnull().values.any() else 'Data doesnt have missing values')
print ('')

# What columns/ variables
print ('Columns with missing values : {}'.format(parcels.columns[parcels.isna().any()].tolist()))
print ('')


#Check features datatypes and see if there is any string feature that needs to be int/float
print(color.BOLD + 'String Features: \n'+ color.END + '{}'.format(parcels.dtypes[parcels.dtypes == "object"]))
print ('')

# Take a count of missing values. What proportion of the data is missing? 
missing_values_count_parcels = parcels[['x', 'y', 'block_id', 'node_id']].isnull().sum()
print(color.BOLD + 'PARCELS' + color.END + '\nCount of missing points: \n {}'.format(missing_values_count_parcels))
print('Proportion of missing points: \n{}'.format(missing_values_count_parcels/ len(parcels.node_id)))

## 4. Node Variables


In [ ]:
# Does data have missing values? 
nodes = orca.get_table('nodes').to_frame()
print(color.RED + 'Data has missing values'+ color.END if nodes.isnull().values.any() else 'Data doesnt have missing values')
print ('')
print ('Nodes table shape{}'.format(nodes.shape))


In [ ]:
#nodes = nodes[(nodes.med_income_500 > 0) & (nodes.med_rent_per_sqft_log > 0)]
#orca.add_table('nodes', nodes)
#print ('Nodes table shape{}'.format(nodes.shape))


In [ ]:
rentals = orca.get_table('rentals').to_frame()

rentals.head()
med_rent = rentals.groupby("node_id").median()
med_rent['price_per_sqft']

# Add this column to node table (they have node_id as their index)
orca.add_column('nodes', 'med_price_per_sqft', med_rent['price_per_sqft'], cache=False, cache_scope='forever')

In [ ]:
nodes = orca.get_table('nodes').to_frame()
nodes.head()
# What columns/ variables
print ('Columns with missing values : {}'.format(nodes.columns[nodes.isna().any()].tolist()))
print ('')


# Take a count of missing values. What proportion of the data is missing? 
missing_values_count_nodes = nodes[['med_price_per_sqft']].isnull().sum()
print(color.BOLD + 'NODES' + color.END + '\nCount of missing points: \n {}'.format(missing_values_count_nodes))
print('Proportion of missing points: \n{}'.format(missing_values_count_nodes/ len(nodes)))

##I don't recommend this method at all. But in case one wants to do it 

nodes.dropna(axis=0, how='any', inplace = True)

## Update the whole table
orca.add_table('nodes', nodes)


In [ ]:
numeric_feats = nodes.dtypes[nodes.dtypes != "object"].index

skewed_feats = nodes[numeric_feats].apply(lambda x: skew(x)) #compute skewness
print(color.BOLD +'Skewed features are as follows'+ color.END + '\n{}'.format (skewed_feats))

In [ ]:
# Only income needs to be normalized
households.income = np.log1p(households.income)

# Update column
orca.get_table('households').update_col_from_series('income', households.income)

In [10]:
df = orca.merge_tables(target='units', tables=['units','buildings','nodes'])

In [ ]:
print(len(units))
print(len(df))

In [ ]:
len (df[df.non_residential_sqft != 0])

In [ ]:
len (df[df.res_price_per_sqft == 0])


In [ ]:
df.redfin_home_type.value_counts()

In [ ]:
df.costar_property_type.value_counts()

In [ ]:
nodes = orca.get_table('nodes').to_frame()
print('{} nodes have median income zero or below'.format(len(nodes[nodes.med_income_500 <= 0])))
print('Total nodes count {}'.format(len (nodes)))



In [ ]:
households= orca.get_table('households').to_frame()
len(households[households.income <= 0])

### Warnings:

Updating a column gives error if you want to convert obj (string) to integer


### To Do:
    
- clean the redfin_home_type var from Buildings table
- check the quality of nodes variables


### Cheat Sheet

In [ ]:
# To add a new column to an existing orca table
#orca.add_column('name_of_the_table', 'new_column_name', new_column, cache=False, cache_scope='forever')